In [1]:
from SPO import *
from SelectiveRegression import *
from LinearRegression import *

In [2]:
(X_1gtrain, y_train, X_1gtest, y_test) = get_energy()

/home/jayanta/Documents/Projects/knapsack/KnapsackSPO/EnergyCost/get_energy.py:23: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  X1g = df.loc[:, df.columns != 'SMPEP2'].as_matrix()
/home/jayanta/Documents/Projects/knapsack/KnapsackSPO/EnergyCost/get_energy.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  y = df.loc[:, 'SMPEP2'].as_matrix()


In [3]:
y_train.shape

(26496,)

In [4]:
y_test.shape

(11376,)

In [7]:
48*14

672

In [5]:
26496//48

552

In [6]:
11376//48

237

In [ ]:
from SelectiveRegression import *
from SPO import *
from LinearRegression import *
import pandas as pd
import numpy as np
(X_1gtrain, y_train, X_1gtest, y_test) = get_energy()

jobs = [4]
no_exp = 10

Models= {"SPO":SGD_SPO,"slereg":Pytorch_SelReg}
lr_list =  [10**(-i) for i in range(1,3)]
performance = []
for m in Models:
	measure_name = ["model","lr" ,"Mean loss","SD loss","Mean regret","SD regret","Mean accuracy","SD accuracy","Mean precision","SD precision","Mean tpr","Mean fpr"]
	for lr in lr_list:
		accuracy_measure =[ ]
		precision_measure =[]
		tpr_measure= []
		fpr_measure = []
		loss_measure= []
		regret_measure= []
		for x in range(no_exp):
			clf= Models[m](jobs= jobs,epochs= 10,lr=lr)
			try:
				clf.fit(X_1gtrain,y_train)
				scr = clf.test_score(X_1gtest,y_test)
				cf = scr['confusion_matrix']
				accuracy = (cf[0,0]+cf[1,1]) /np.sum(cf)
				tpr =  cf[0,0]/(cf[0,0]+cf[1,0])
				fpr = cf[1,0]/(cf[1,0]+cf[1,1])
				precision = cf[0,0]/(cf[0,0]+cf[0,1]) 
				regret_measure.append(scr['regret'])
				accuracy_measure.append(accuracy)
				tpr_measure.append(tpr)
				fpr_measure.append(fpr)
				precision_measure.append(precision)
				loss_measure.append(scr['loss'])
			except Exception as e:
				print("type error: " + str(e))
			clf.fit(X_1gtrain,y_train)
			scr = clf.test_score(X_1gtest,y_test)
			cf = scr['confusion_matrix']
			accuracy = (cf[0,0]+cf[1,1]) /np.sum(cf)
			tpr =  cf[0,0]/(cf[0,0]+cf[1,0])
			fpr = cf[1,0]/(cf[1,0]+cf[1,1])
			precision = cf[0,0]/(cf[0,0]+cf[0,1]) 
			regret_measure.append(scr['regret'])
			accuracy_measure.append(accuracy)
			tpr_measure.append(tpr)
			fpr_measure.append(fpr)
			precision_measure.append(precision)
			loss_measure.append(scr['loss'])
		performance.append( [m,lr,np.mean(loss_measure),np.std(loss_measure),np.mean(regret_measure),np.std(regret_measure),np.mean(accuracy_measure),np.std(accuracy_measure),np.mean(precision_measure),np.std(precision_measure),\
		np.mean(tpr_measure),np.mean(fpr_measure)])

performance = pd.DataFrame(performance,columns= measure_name)
performance.to_csv("ExperimentResults_5.csv")


In [18]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
import torch
from torch import nn, optim
from torch.autograd import Variable
from get_energy import get_energy
from energy_cost import *

class Pytorch_regression:
    def __init__(self,jobs, epochs=2, doScale=True, n_items=48, verbose=False,plotting=False,return_regret= False,optimizer= optim.SGD,**hyperparam):
        self.n_items = n_items
        self.jobs = jobs

        self.hyperparam = hyperparam
        self.epochs = epochs

        self.doScale=doScale
        self.verbose=verbose
        self.plotting = plotting
        self.return_regret = return_regret
        self.optimizer = optimizer

        self.scaler = None
        self.model = None
        self.best_params_ = {"p":"default"}
        self.criterion = nn.MSELoss()


    def fit(self, x_train, y_train,x_validation=None,y_validation=None,batch=False):
        qids = np.array(x_train[:,0], dtype=int) # qid column
        x_train = x_train[:,1:] # without group ID
        validation = (x_validation is not None) and (y_validation is not None)
        
        # scale data?
        if self.doScale:
            self.scaler = preprocessing.StandardScaler().fit(x_train)
            x_train = self.scaler.transform(x_train)

        trch_X_train = torch.from_numpy(x_train).float()
        trch_y_train = torch.from_numpy(np.array([y_train]).T).float()
        
        if self.plotting:
            subepoch_list= []
            loss_list =[]
            regret_list = []
            if validation:
                loss_list_validation= []
                regret_list_validation= []
        # basics
        jobs = self.jobs
        n_items = self.n_items
        n_knapsacks = len(trch_X_train)//n_items
        # prepping
        knaps_V_true = [get_profits(trch_y_train, kn_nr, n_items) for kn_nr in range(n_knapsacks)]
        knaps_sol = [get_energy_indicators(V_true,jobs) for V_true in knaps_V_true]
        
        # network
        self.model = LinearRegression(trch_X_train.shape[1],1) # input dim, output dim

        # loss
        
        optimizer = self.optimizer(self.model.parameters(), **self.hyperparam)
        num_epochs = self.epochs

        # training
        subepoch = 0 # for logging and nice curves
        logger = [] # (dict_epoch, dict_train, dict_test)
        for epoch in range(num_epochs):
            #scheduler.step()
            knapsack_nrs = [x for x in range(n_knapsacks)]
            random.shuffle(knapsack_nrs) # randomly shuffle order of training
            cnt=0
            if not batch:
                train_fwdbwd(self.model, self.criterion, optimizer, trch_X_train, trch_y_train, 1.0)
            else:
                for kn_nr in knapsack_nrs:
                    V_true = knaps_V_true[kn_nr]
                    kn_start = kn_nr*n_items
                    kn_stop = kn_start+n_items
                    suby=  torch.from_numpy(V_true).float()
                    train_fwdbwd(model= self.model, criterion=self.criterion, 
                             optimizer= optimizer, sub_X_train=trch_X_train[kn_start:kn_stop],
                             sub_y_train= suby, mult=1.0)
            if validation:
                    dict_validation = self.test_score(x_validation,y_validation)
            if self.plotting:
                    cnt += 1
                    subepoch += 1
                    dict_train = test_fwd(self.model, self.criterion, trch_X_train, trch_y_train, n_items, self.jobs)
                    loss_list.append(dict_train['loss'])
                    regret_list.append(dict_train['regret'])
                    subepoch_list.append(subepoch)
                    if validation:
                        loss_list_validation.append(dict_validation['loss'])
                        regret_list_validation.append(dict_validation['regret'])
            if self.verbose:
                        if not self.plotting:
                            cnt += 1
                            subepoch += 1
                        if cnt :
                            dict_epoch = {'epoch': epoch+1, 'subepoch': subepoch, 'cnt': cnt}
                            dict_train = test_fwd(self.model, self.criterion, trch_X_train, trch_y_train, n_items, self.jobs)
                            if validation:
                                logger.append( (dict_epoch, dict_train,dict_validation) )
                                print('Epoch[{}/{}]::{}, loss(train): {:.6f}, regret(train): {:.2f}, loss(validation): {:.6f}, regret(validation): {:.2f}'.format(epoch+1, 
                                    num_epochs, cnt, dict_train['loss'], dict_train['regret'],dict_validation['loss'],dict_validation['regret']  ))
                            else:
                                logger.append( (dict_epoch, dict_train) )
                                print('Epoch[{}/{}]::{}, loss: {:.6f}, regret(train): {:.2f}'.format(epoch+1, num_epochs, cnt, dict_train['loss'], dict_train['regret']))
        
        if self.plotting:
            import matplotlib.pyplot as plt
            if validation:
                plt.subplot(2, 1, 1)
                plt.plot(subepoch_list,regret_list,subepoch_list,regret_list_validation)
                plt.title('Learning Curve')
                plt.ylabel('Regret')
                plt.ylim(top=  np.mean(regret_list)+ 5*np.std(regret_list))
                plt.legend(["training","validation"])
                plt.subplot(2, 1, 2)
                plt.plot(subepoch_list, loss_list,subepoch_list,loss_list_validation)
                plt.xlabel('Sub Epochs')
                plt.ylabel('Loss')
                plt.ylim(top= np.median(loss_list)+ 5*np.std(loss_list))
                plt.legend(["training","validation"])
                plt.show()
            else:
                plt.subplot(2, 1, 1)
                plt.plot(subepoch_list,regret_list)
                plt.title('Learning Curve')
                plt.ylabel('Regret')
                plt.ylim(top= np.mean(regret_list)+ 5*np.std(regret_list))
                plt.subplot(2, 1, 2)
                plt.plot(subepoch_list, loss_list)
                plt.ylim(top= np.median(loss_list)+ 5*np.std(loss_list))
                plt.xlabel('Sub Epochs')
                plt.ylabel('Loss')
                plt.show()        
        if self.return_regret:
            dict_train = test_fwd(self.model, self.criterion, trch_X_train, trch_y_train, n_items, self.jobs)
            if validation:
                dict_validation = self.test_score(x_validation,y_validation)
                return {"loss_training":dict_train['loss'].item(),"regret_training":dict_train['regret'],
                "loss_validation":dict_validation['loss'],"regret_validation":dict_validation['regret']}
            
            return {"loss":dict_train['loss'].item(),"regret":dict_train['regret']}

    def predict(self, x_test):
        qids = np.array(x_test[:,0], dtype=int) # qid column
        x_test = x_test[:,1:] # drop qid column
        
        # scale data?
        if self.doScale:
            x_test = self.scaler.transform(x_test)

        trch_X = torch.from_numpy(x_test).float()
        pred = self.model(Variable(trch_X))
        return pred.data.numpy().T[0] # as numpy array (transpose)
    def predit_knapsack(self, x_test):
        qids = np.array(x_test[:,0], dtype=int) # qid column
        x_test = x_test[:,1:] # drop qid column

        # scale data?
        if self.doScale:
            x_test = self.scaler.transform(x_test)

        trch_X = torch.from_numpy(x_test).float()
        n_items = self.n_items
        n_knapsacks = len(trch_X)//n_items
        capacity = self.capacity

        knapsack_nrs = [x for x in range(n_knapsacks)]
        pred = []
        for kn_nr in knapsack_nrs:
            V_pred = get_profits_pred(self.model, trch_X, kn_nr, n_items)
            sol_pred = get_kn_indicators(V_pred, capacity, weights=self.weights,use_dp=True)
            pred.append(sol_pred)
        return np.array(pred)

    def test_score(self,x_test,y_test):
        qids = np.array(x_test[:,0],dtype=int)
        x_test = x_test[:,1:] # drop qid column
        # scale data?
        if self.doScale:
            x_test = self.scaler.transform(x_test)
        trch_X = torch.from_numpy(x_test).float()
        trch_y = torch.from_numpy(np.array([y_test]).T).float()
        n_items = self.n_items
        n_knapsacks = len(trch_X)//n_items
 
        dict_test = test_fwd(self.model, self.criterion, trch_X, trch_y, n_items, self.jobs)

        return {"loss":dict_test['loss'].item(),"regret":dict_test['regret'],"confusion_matrix": dict_test['confusion_matrix']}


In [1]:
import time
from datetime import datetime

In [2]:
import itertools
(X_1gtrain, y_train, X_1gtest, y_test) = get_energy()

NameError: name 'get_energy' is not defined

In [7]:
X_1gt.shape

(11376, 9)

In [21]:
clf = Pytorch_regression(jobs= [4],epochs=10,lr=0.1)

In [20]:
clf.model.parameters

AttributeError: 'NoneType' object has no attribute 'parameters'

In [9]:
price= [449526937600000000, 425062195200000000, 407515673600000000, 392807424000000000, 380355686400000000, 375180288000000000, 371155404800000000, 368776371200000000, 369282329600000000, 371904640000000000, 375749580800000000, 385961062400000000, 404509388800000000, 441566105600000000, 484679065600000000, 531735296000000000, 567081267200000000, 584651110400000000, 604333056000000000, 622673356800000000, 632507392000000000, 638169907200000000, 646912921600000000, 653866393600000000, 662161715200000000, 669212876800000000, 666866739200000000, 654706329600000000, 657838899200000000, 664836403200000000, 670313011200000000, 677257574400000000, 687890432000000000, 712615628800000000, 752232857600000000, 786283673600000000, 784784793600000000, 767039795200000000, 742730393600000000, 726295756800000000, 698675507200000000, 676215296000000000, 659768473600000000, 627737292800000000, 601412198400000000, 569674752000000000, 539412480000000000, 522785792000000000]

price = [p/1e+8 for p in price]
cost_minimize(price,[4])

ERROR HAPPENED!

[449526937600000000, 425062195200000000, 407515673600000000, 392807424000000000, 380355686400000000, 375180288000000000, 371155404800000000, 368776371200000000, 369282329600000000, 371904640000000000, 375749580800000000, 385961062400000000, 404509388800000000, 441566105600000000, 484679065600000000, 531735296000000000, 567081267200000000, 584651110400000000, 604333056000000000, 622673356800000000, 632507392000000000, 638169907200000000, 646912921600000000, 653866393600000000, 662161715200000000, 669212876800000000, 666866739200000000, 654706329600000000, 657838899200000000, 664836403200000000, 670313011200000000, 677257574400000000, 687890432000000000, 712615628800000000, 752232857600000000, 786283673600000000, 784784793600000000, 767039795200000000, 742730393600000000, 726295756800000000, 698675507200000000, 676215296000000000, 659768473600000000, 627737292800000000, 601412198400000000, 569674752000000000, 539412480000000000, 522785792000000000]


In [22]:
clf.fit(X_1gtrain,y_train)

In [23]:
clf.predict(X_1gtrain)

array([44.799522, 42.654385, 40.945137, ..., 42.99395 , 39.673035,
       29.979073], dtype=float32)

In [29]:
clf.test_score(X_1gtest,y_test)

ValueError: cannot convert float NaN to integer

In [6]:
SPO = SGD_SPO(jobs=[4], epochs=10,verbose=True,lr= 1e-3)

In [ ]:
SPO.fit(X_1gtrain,y_train)

Epoch[1/10]::50, loss: 63707848704.000000, regret(train): 16.29
Epoch[1/10]::100, loss: 162455846912.000000, regret(train): 16.23
Epoch[1/10]::150, loss: 386045444096.000000, regret(train): 16.24
Epoch[1/10]::200, loss: 594717900800.000000, regret(train): 16.23
Epoch[1/10]::250, loss: 859895824384.000000, regret(train): 16.21
Epoch[1/10]::300, loss: 1189212979200.000000, regret(train): 16.26
Epoch[1/10]::350, loss: 1806892138496.000000, regret(train): 16.24
Epoch[1/10]::400, loss: 2351595520000.000000, regret(train): 16.15
Epoch[1/10]::450, loss: 3169707884544.000000, regret(train): 16.21
Epoch[1/10]::500, loss: 3817445785600.000000, regret(train): 16.21
Epoch[1/10]::550, loss: 4749238206464.000000, regret(train): 16.28
Epoch[2/10]::50, loss: 5202888884224.000000, regret(train): 16.24
Epoch[2/10]::100, loss: 6359834689536.000000, regret(train): 16.15
Epoch[2/10]::150, loss: 7378356928512.000000, regret(train): 16.21
Epoch[2/10]::200, loss: 8180350844928.000000, regret(train): 16.26
Epo

In [1]:
SPO.test_score(X_1gtest,y_test)

NameError: name 'SPO' is not defined

In [1]:
from energy_cost import *

In [15]:
get_energy_indicators(y_train[103:184],[6])

array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [3]:
profits = [1, 2, 3, 4, 5, 6, 7, 8, 9]

weights = [ [1, 2, 3, 4, 5, 6, 7, 8, 9],
                                 [1, 1, 1, 1, 1, 1, 1, 1, 1]]
capacities = [34, 4]

In [7]:
from ortools.algorithms import pywrapknapsack_solver
solver= pywrapknapsack_solver.KnapsackSolver(pywrapknapsack_solver.KnapsackSolve.KNAPSACK_MULTIDIMENSION_SOLVER,
                         "Multi-dimensional solver");

AttributeError: module 'ortools.algorithms.pywrapknapsack_solver' has no attribute 'KnapsackSolve'

In [11]:
solver = pywrapknapsack_solver.KnapsackSolver(
      pywrapknapsack_solver.KnapsackSolver.
       KNAPSACK_MULTIDIMENSION_CBC_MIP_SOLVER,
      'test')

In [17]:
solver.Init(profits, weights, capacities)
computed_value = solver.Solve()


In [18]:
computed_value

30

In [15]:
[int(solver.BestSolutionContains(x)) for x in range(len(profits))]

[0, 0, 0, 0, 0, 1, 1, 1, 1]

In [28]:
[10**(-i) for i in range(1,7)]

[0.1, 0.01, 0.001, 0.0001, 1e-05]

In [26]:
1e-2

0.01

In [4]:
y = [i for i in range(48)]

In [8]:
cost_minimize(y,[4,6])

{'objective': 21.0,
 'assignments': [[1,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [1,
   1,
   1,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0]]}

In [63]:
class Gurobi_ICON:
    def __init__(self,nbMachines,nbTasks,nbResources,MC,U,D,E,L,P,idle,up,down,q,relax=False,
        verbose=False,method=-1,obj_cut=-1,**h):
        self.nbMachines  = nbMachines
        self.nbTasks = nbTasks
        self.nbResources = nbResources
        self.MC = MC
        self.U =  U
        self.D = D
        self.E = E
        self.L = L
        self.P = P
        self.idle = idle
        self.up = up
        self.down = down
        self.q= q
        self.relax = relax
        self.verbose = verbose
        self.method = method
        self.obj_cut = obj_cut 
        self.sol_hist = []
        #-1 : no objective cut; 0: cut for predictions only 'true' solution; 1: use sol_hist
        
    def make_model(self):
        Machines = range(self.nbMachines)
        Tasks = range(self.nbTasks)
        Resources = range(self.nbResources)

        MC = self.MC
        U =  self.U
        D = self.D
        E = self.E
        L = self.L
        P = self.P
        idle = self.idle
        up = self.up
        down = self.down
        relax = self.relax
        q= self.q
        N = 1440//q

        M = Model("icon")
        if not self.verbose:
            M.setParam('OutputFlag', 0)
        if relax:
            x = M.addVars(Tasks, Machines, range(N), lb=0., ub=1., vtype=GRB.CONTINUOUS, name="x")
        else:
            x = M.addVars(Tasks, Machines, range(N), vtype=GRB.BINARY, name="x")

        # x = {}
        # for f in Tasks:
        #     for m in Machines:
        #         for t in range(N):
        #             if relax:
        #                 x[(f,m,t)] = M.addVar(0.,1.,name= "x"+str(f)+"_"+str(m)+"_"+str(t))
        #             else:
        #                 x[(f,m,t)] = M.addVar(vtype=GRB.BINARY,name= "x"+str(f)+"_"+str(m)+"_"+str(t))
                    
        # earliest start time constraint
        #M.addConstrs( (x[(f,m,t)]==0 )  for f in Tasks for m in Machines for t in range(E[f]) )
        # latest end time constraint
        #M.addConstrs( (x[(f,m,t)]==0 )   for f in Tasks for m in Machines for t in range(L[f]-D[f] +1,N) )
        M.addConstrs( x.sum(f,'*',range(E[f])) == 0 for f in Tasks)
        M.addConstrs( x.sum(f,'*',range(L[f]-D[f]+1,N)) == 0 for f in Tasks)



        M.addConstrs(( quicksum(x[(f,m,t)] for t in range(N) for m in Machines) == 1  for f in Tasks))

        # capacity requirement
        for r in Resources:
            for m in Machines:
                for t in range(N):
                    M.addConstr( quicksum( quicksum(x[(f,m,t1)]  for t1 in range(max(0,t-D[f]+1),t+1) )*
                                   U[f][r] for f in Tasks) <= MC[m][r])   
        M.update()
        self.model = M
        self.x = x 

    def solve_model(self,price,scheduling=False,warmstart=None,timelimit=None):
        Model = self.model
        MC = self.MC
        U =  self.U
        D = self.D
        E = self.E
        L = self.L
        P = self.P
        idle = self.idle
        up = self.up
        down = self.down
        q= self.q
        N = 1440//q  
        newcut = None
        Model.reset()

        verbose = self.verbose
        x =  self.x
        nbMachines = self.nbMachines
        nbTasks = self.nbTasks
        nbResources = self.nbResources
        Machines = range(nbMachines)
        Tasks = range(nbTasks)
        Resources = range(nbResources)
        obj_expr = quicksum( (quicksum(x[f,m,t] for m in Machines)*np.sum(price[t:t+D[f]])*P[f]*q/60) 
            for f in Tasks for t in range(N-D[f]+1))

        bestval = None # MINIMISATION problem!
        if warmstart is not None and (self.obj_cut==0):
            (pvars,dcons,solt) = warmstart
            bestval = np.sum(solt*price)

            # OLD: add Pstart/Dstart, not sure this works, make sure presolve still runs!
            '''for i,var in enumerate(Model.getVars()):
                var.Pstart = pvars[i]
            for i,cons in enumerate(Model.getConstrs()):
                cons.Dstart = dcons[i]'''
        if (self.obj_cut==1) and (len(self.sol_hist) > 0):
            for sol in self.sol_hist:
                 val = np.sum(sol*price)
                 if (bestval is None) or (val < bestval):
                    bestval = val
        if bestval is not  None:
            # Minimization; make sure to remove the cut again below!!
            newcut = Model.addConstr(obj_expr <= (bestval+1e-3), name="objcut")                            
        
        Model.setObjective(obj_expr, GRB.MINIMIZE)
        #Model.setObjective( quicksum( (x[(f,m,t)]*P[f]*quicksum([price[t+i] for i in range(D[f])])*q/60) for f in Tasks
        #                for m in Machines for t in range(N-D[f]+1)), GRB.MINIMIZE)
        if timelimit:
            Model.setParam('TimeLimit', timelimit)
        #if relax:
        #    Model = Model.relax()
        Model.setParam('Method', self.method)
        logging.info("Number of constraints%d",Model.NumConstrs)
        Model.optimize()

        solver = np.zeros(N)
        if Model.status in [GRB.Status.OPTIMAL,9]:
           
                #task_on = Model.getAttr('x',x)
                task_on = np.zeros( (nbTasks,nbMachines,N) )
                for f in Tasks:
                    for m in Machines:
                        for t in range(N):
                            task_on[f,m,t] = x[f,m,t].X

                if verbose:
                    for k,val in task_on.items():
                        if int(val)>0:
                            print("Task_%d starts on machine_%d at %d"%(k[0],k[1],k[2]))
                    print('\nCost: %g' % Model.objVal)
                    print('\nExecution Time: %f' %Model.Runtime)

                # remove cut again (modifies model)
                if newcut is not None:
                    Model.remove(newcut)
                    newcut = None
                
                for t in range(N):        
                    solver[t] = np.sum( np.sum(task_on[f,:,max(0,t-D[f]+1):t+1])*P[f] for f in Tasks )  
                solver = solver*q/60
                
                print(sum( (sum(x[f,m,t].X for m in Machines)*np.sum(price[t:t+D[f]])*P[f]*q/60) 
            for f in Tasks for t in range(N-D[f]+1)))
                print(Model.objVal)
         
                
                schedule = np.zeros((nbTasks,nbMachines,N))
                for f in Tasks:
                    for m in Machines:
                        for t in range(N):
                                 schedule[f,m,t] = task_on[(f,m,t)]
                return solver,schedule
                

        elif Model.status == GRB.Status.INF_OR_UNBD:
            print('Model is infeasible or unbounded')
        elif Model.status == GRB.Status.INFEASIBLE:
            print('Model is infeasible')
        elif Model.status == GRB.Status.UNBOUNDED:
            print('Model is unbounded')
        else:
            print('Optimization ended with status %d' % Model.status)

        # make sure cut is removed! (modifies model)
        if newcut is not None:
            Model.remove(newcut)

        return solver,Model.Runtime   


In [118]:
param = data_reading("load2/day13.txt")
Machines = range(param['nbMachines'])
Tasks = range(param['nbTasks'])
Resources = range(param['nbResources'])

D = param['D']
P = param['P']
price = y_train[96:144]
q = param['q']
N = 1440//q

In [119]:
clf_train =  Gurobi_ICON(**param)
clf_train.make_model()
sol = clf_train.solve_model(price)

1661102.7726500004
1661102.7726500002


/home/jayanta/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:160: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


In [120]:
sum(sol[0]*price)

1661102.7726500002

In [121]:
x = sol[1]

In [129]:
print(sum( (sum(x[f,m,t] for m in Machines)*np.sum(price[t:t+D[f]])*P[f]*q/60) 
            for f in Tasks for t in range(N-D[f]+1)))

1661102.7726500004


In [131]:
sum( (np.sum(x[f,m,t] for m in Machines)*np.sum(price[t:t+D[f]])*P[f]*q/60) 
            for f in Tasks for t in range(N-D[f]+1))

/home/jayanta/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  


1661102.7726500004